In [3]:
#Loading the libraries 

import pandas as pd
import numpy as np
#import xgboost as xgb
#from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

C:\Users\my\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#ignore this
#reading the file containing the review with the ratings
list_of_review = []
with open('amazon_cells_labelled.txt','r',encoding = 'utf-8') as file:
            mylist = [line.rstrip('\n') for line in file]
for each in mylist:
    list_of_review.append(each.split('\t'))

In [3]:
#Ignore this too
#converting it into a dataframe

df = pd.DataFrame([i[0] for i in list_of_review], columns=['Review'])
df['sentiment'] = [i[1] for i in list_of_review]
print(df)

                                                Review sentiment
0    So there is no way for me to plug it in here i...         0
1                          Good case, Excellent value.         1
2                               Great for the jawbone.         1
3    Tied to charger for conversations lasting more...         0
4                                    The mic is great.         1
5    I have to jiggle the plug to get it to line up...         0
6    If you have several dozen or several hundred c...         0
7          If you are Razr owner...you must have this!         1
8                  Needless to say, I wasted my money.         0
9                     What a waste of money and time!.         0
10                     And the sound quality is great.         1
11   He was very impressed when going from the orig...         1
12   If the two were seperated by a mere 5+ ft I st...         0
13                            Very good quality though         1
14   The design is very o

In [7]:
#Loading the dataset

df =  pd.read_csv('data_updated.csv')

In [8]:
df.head()

,Review,sentiment
0,So there is no way for me to plug it in here i...,0.1
1,"Good case, Excellent value.",1.3
2,Great for the jawbone.,1.2
3,Tied to charger for conversations lasting more...,0.2
4,The mic is great.,1.2


In [9]:
#checking for any empty rows
df=df[~df['Review'].isnull()]

In [10]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(df['sentiment'])

In [12]:
# y is encoded on a scale of 0 to 5
y

array([0, 5, 4, 1, 4, 1, 1, 5, 2, 2, 4, 5, 0, 4, 1, 5, 2, 4, 4, 0, 0, 5,
       0, 5, 1, 4, 5, 0, 1, 1, 1, 4, 1, 2, 5, 1, 3, 2, 4, 2, 4, 0, 4, 1,
       5, 3, 2, 0, 3, 0, 0, 3, 3, 3, 5, 3, 1, 4, 4, 0, 1, 5, 5, 1, 0, 0,
       1, 3, 4, 4, 0, 1, 4, 3, 3, 5, 2, 4, 5, 2, 4, 0, 5, 1, 1, 5, 4, 0,
       5, 5, 1, 3, 4, 0, 1, 5, 2, 0, 5, 2, 3, 2, 3, 4, 1, 4, 1, 5, 1, 1,
       5, 4, 3, 4, 5, 3, 2, 4, 5, 3, 4, 1, 4, 0, 3, 4, 1, 1, 4, 0, 3, 5,
       4, 5, 1, 1, 4, 3, 0, 5, 1, 2, 1, 2, 1, 4, 2, 1, 1, 2, 0, 1, 4, 4,
       1, 5, 3, 5, 5, 5, 0, 4, 0, 4, 1, 4, 1, 4, 0, 0, 5, 1, 0, 4, 4, 4,
       3, 1, 0, 1, 0, 1, 5, 2, 1, 5, 0, 4, 0, 1, 1, 4, 3, 1, 0, 3, 5, 1,
       0, 1, 4, 4, 1, 5, 0, 4, 3, 4, 4, 1, 0, 0, 4, 3, 3, 5, 5, 0, 2, 5,
       1, 1, 4, 1, 1, 4, 3, 1, 1, 2, 1, 0, 1, 4, 3, 5, 5, 4, 4, 1, 0, 2,
       2, 0, 1, 5, 4, 4, 0, 5, 5, 5, 4, 5, 1, 1, 0, 4, 5, 4, 4, 3, 3, 2,
       0, 3, 1, 3, 1, 3, 1, 3, 2, 5, 1, 4, 1, 4, 5, 4, 4, 5, 0, 5, 3, 4,
       1, 1, 0, 4, 4, 1, 5, 2, 5, 5, 4, 3, 2, 1, 0,

In [13]:
#splitting the data into train and test
xtrain, xvalid, ytrain, yvalid = train_test_split(df['Review'], y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2)

In [14]:
#converting the text into count vector matrix

ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [15]:
#Implementing Logistic Regression with the count vector matrix

clf = LogisticRegression(penalty='l2',C=1.0)
clf.fit(xtrain_ctv, ytrain)
y_pred = clf.predict(xvalid_ctv)
print("Accuracy: {}".format(clf.score(xvalid_ctv, yvalid)))

Accuracy: 0.53


In [16]:
#Fitting Naive bayes with count vector matrix

clf = MultinomialNB(alpha=0.1)
clf.fit(xtrain_ctv, ytrain)
print("Accuracy: {}".format(clf.score(xvalid_ctv, yvalid)))

Accuracy: 0.385


In [17]:
#converting the text into Tfidf Vector

tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [18]:
#Fitting Logistic Regression with the Tfidfvector

clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
print("Accuracy: {}".format(clf.score(xvalid_tfv, yvalid)))

Accuracy: 0.53


In [19]:
#Fitting Naive bayes with the Tfidfvector

clf = MultinomialNB(alpha=0.1)
clf.fit(xtrain_tfv, ytrain)
print("Accuracy: {}".format(clf.score(xvalid_tfv, yvalid)))

Accuracy: 0.44


In [13]:
import os
# load the GloVe vectors in a dictionary:
os.chdir("C:\\Users\\Desktop")
embeddings_index = {}
failed_words= []
f = open('glove.840B.300d.txt',encoding='utf-8')
for line in tqdm(f):
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        values = line.split()
        word = values[0]     
        failed_words.append(word)
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [03:07, 11712.21it/s]


Found 2195884 word vectors.


In [14]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [15]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 5391.14it/s]


In [20]:
# using keras tokenizer here

token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences

xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [21]:
word_index

{'the': 1,
 'i': 2,
 'and': 3,
 'it': 4,
 'is': 5,
 'a': 6,
 'this': 7,
 'to': 8,
 'phone': 9,
 'my': 10,
 'for': 11,
 'of': 12,
 'not': 13,
 'with': 14,
 'very': 15,
 'great': 16,
 'on': 17,
 'was': 18,
 'in': 19,
 'that': 20,
 'good': 21,
 'have': 22,
 'you': 23,
 'product': 24,
 'quality': 25,
 'had': 26,
 'works': 27,
 'headset': 28,
 'battery': 29,
 'but': 30,
 'as': 31,
 'sound': 32,
 'so': 33,
 'are': 34,
 'well': 35,
 'use': 36,
 'all': 37,
 'one': 38,
 'ear': 39,
 'would': 40,
 'has': 41,
 'work': 42,
 'from': 43,
 'your': 44,
 'like': 45,
 'be': 46,
 'case': 47,
 'if': 48,
 'me': 49,
 'than': 50,
 "i've": 51,
 "don't": 52,
 'excellent': 53,
 'up': 54,
 'after': 55,
 'time': 56,
 'price': 57,
 "it's": 58,
 'no': 59,
 'really': 60,
 'recommend': 61,
 '2': 62,
 'does': 63,
 'at': 64,
 'out': 65,
 'service': 66,
 'or': 67,
 'only': 68,
 'best': 69,
 'get': 70,
 "i'm": 71,
 'also': 72,
 'when': 73,
 'nice': 74,
 'just': 75,
 'too': 76,
 'worked': 77,
 'love': 78,
 'am': 79,
 'thes

In [20]:
# create an embedding matrix for the words we have in the dataset

embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████████████████████████████████████████████████████████████████████| 1878/1878 [00:00<00:00, 208240.44it/s]


In [16]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [17]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [18]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [40]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300,dropout=0.1, recurrent_dropout=0.3))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(2))
model.add(Activation('softmax'))
from keras.optimizers import Adam

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [35]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=3, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=32, epochs=20, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - ETA: 37s - loss: 0.7124 - acc: 0.46 - ETA: 19s - loss: 0.7249 - acc: 0.39 - ETA: 13s - loss: 0.7106 - acc: 0.42 - ETA: 9s - loss: 0.7117 - acc: 0.4297 - ETA: 7s - loss: 0.7172 - acc: 0.412 - ETA: 6s - loss: 0.7095 - acc: 0.437 - ETA: 5s - loss: 0.7047 - acc: 0.450 - ETA: 4s - loss: 0.7008 - acc: 0.476 - ETA: 4s - loss: 0.6968 - acc: 0.493 - ETA: 3s - loss: 0.6930 - acc: 0.506 - ETA: 3s - loss: 0.6894 - acc: 0.534 - ETA: 2s - loss: 0.6859 - acc: 0.552 - ETA: 2s - loss: 0.6834 - acc: 0.562 - ETA: 2s - loss: 0.6805 - acc: 0.564 - ETA: 2s - loss: 0.6779 - acc: 0.568 - ETA: 1s - loss: 0.6756 - acc: 0.570 - ETA: 1s - loss: 0.6739 - acc: 0.575 - ETA: 1s - loss: 0.6666 - acc: 0.588 - ETA: 1s - loss: 0.6593 - acc: 0.600 - ETA: 0s - loss: 0.6518 - acc: 0.612 - ETA: 0s - loss: 0.6464 - acc: 0.617 - ETA: 0s - loss: 0.6399 - acc: 0.626 - ETA: 0s - loss: 0.6363 - acc: 0.635 - ETA: 0s - loss: 0.6481 - 

In [50]:
# create the model
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.4))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 70, 300)           563700    
_________________________________________________________________
spatial_dropout1d_13 (Spatia (None, 70, 300)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 70, 32)            28832     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 35, 32)            0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 1120)              0         
_________________________________________________________________
dense_35 (Dense)             (None, 250)               280250    
_________________________________________________________________
dropout_18 (Dropout)         (None, 250)               0         
__________

In [51]:
# Fit the model
model.fit(xtrain_pad, ytrain_enc, validation_data=(xvalid_pad, yvalid_enc), epochs=20, batch_size=32, verbose=2)
# Final evaluation of the model
scores = model.evaluate(xvalid_pad, yvalid_enc, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 800 samples, validate on 200 samples
Epoch 1/20
 - 1s - loss: 0.7207 - acc: 0.5087 - val_loss: 0.6799 - val_acc: 0.6200
Epoch 2/20
 - 0s - loss: 0.6821 - acc: 0.5862 - val_loss: 0.6614 - val_acc: 0.6775
Epoch 3/20
 - 0s - loss: 0.6373 - acc: 0.6687 - val_loss: 0.5922 - val_acc: 0.7675
Epoch 4/20
 - 0s - loss: 0.5492 - acc: 0.7437 - val_loss: 0.4948 - val_acc: 0.7925
Epoch 5/20
 - 0s - loss: 0.4548 - acc: 0.7944 - val_loss: 0.4630 - val_acc: 0.7925
Epoch 6/20
 - 0s - loss: 0.3844 - acc: 0.8319 - val_loss: 0.4609 - val_acc: 0.7725
Epoch 7/20
 - 0s - loss: 0.3289 - acc: 0.8719 - val_loss: 0.4647 - val_acc: 0.7625
Epoch 8/20
 - 0s - loss: 0.2747 - acc: 0.8875 - val_loss: 0.4402 - val_acc: 0.7900
Epoch 9/20
 - 0s - loss: 0.2326 - acc: 0.9106 - val_loss: 0.4544 - val_acc: 0.8075
Epoch 10/20
 - 0s - loss: 0.2211 - acc: 0.9156 - val_loss: 0.4734 - val_acc: 0.7750
Epoch 11/20
 - 0s - loss: 0.1739 - acc: 0.9369 - val_loss: 0.5041 - val_acc: 0.7825
Epoch 12/20
 - 0s - loss: 0.1635 - acc: